In [1]:
## This file is for testing using FMP as opposed to IEX for accessing historical data for stock prices 

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

import quandl

from dotenv import load_dotenv
load_dotenv()

True

In [11]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

In [5]:
start_date = '2020-01-01'
end_date = '2021-10-29'

In [7]:
test_symbol = 'GME'
stock_symbol = test_symbol

In [ ]:
## https://fmpcloud.io/api/v3/historical-price-full/AAPL?from=2018-03-12&to=2019-03-12&apikey=64522afe231fb8989717ff4643b36c43

In [13]:
url_hist_price = fmpbase_urlv3+'historical-price-full/'
url_hist_query_with_date = url_hist_price+stock_symbol+'?from='+start_date+'&to='+end_date+'&apikey='+api_key
url_hist_query_with_date

'https://fmpcloud.io/api/v3/historical-price-full/GME?from=2020-01-01&to=2021-10-29&apikey=64522afe231fb8989717ff4643b36c43'

In [14]:
resp_data = requests.get(url_hist_query_with_date)
resp_data

<Response [200]>

In [15]:
df = pd.DataFrame(resp_data.json())
df

,symbol,historical
0,GME,"{'date': '2021-10-29', 'open': 182.81, 'high':..."
1,GME,"{'date': '2021-10-28', 'open': 175.16, 'high':..."
2,GME,"{'date': '2021-10-27', 'open': 180.0, 'high': ..."
3,GME,"{'date': '2021-10-26', 'open': 173.36, 'high':..."
4,GME,"{'date': '2021-10-25', 'open': 169.42, 'high':..."
...,...,...
457,GME,"{'date': '2020-01-08', 'open': 5.49, 'high': 5..."
458,GME,"{'date': '2020-01-07', 'open': 5.77, 'high': 5..."
459,GME,"{'date': '2020-01-06', 'open': 5.8, 'high': 5...."
460,GME,"{'date': '2020-01-03', 'open': 6.21, 'high': 6..."


In [19]:
## Values needed 
## date open high low close volume change changePercent 
json_ = resp_data.json()
#json_

In [21]:
json_['historical'][0] # 0 works as index

{'date': '2021-10-29',
 'open': 182.81,
 'high': 185.75,
 'low': 178.0,
 'close': 183.51,
 'adjClose': 183.51,
 'volume': 2293970.0,
 'unadjustedVolume': 2293970.0,
 'change': 0.7,
 'changePercent': 0.383,
 'vwap': 182.42,
 'label': 'October 29, 21',
 'changeOverTime': 0.00383}

In [24]:
len(json_['historical'])

462

In [50]:
data = json_['historical']
df = pd.DataFrame(data)
df.rename(columns={'date':'Date'},inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
df.set_index('Date',inplace=True)
df.drop(columns='label',inplace=True)
df

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime
Date,,,,,,,,,,,
2020-01-02,6.14,6.47,6.07,6.31,6.31,4453598.0,4453598.0,0.17,2.769,6.28333,0.02769
2020-01-03,6.21,6.25,5.84,5.88,5.88,3543933.0,3543933.0,-0.33,-5.314,5.99000,-0.05314
2020-01-06,5.80,5.91,5.60,5.85,5.85,3394774.0,3394774.0,0.05,0.862,5.78667,0.00862
2020-01-07,5.77,5.83,5.44,5.52,5.52,5230265.0,5230265.0,-0.25,-4.333,5.59667,-0.04333
2020-01-08,5.49,5.85,5.41,5.72,5.72,5629442.0,5629442.0,0.23,4.189,5.66000,0.04189
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,169.42,174.80,167.26,173.97,173.97,1442806.0,1442806.0,4.55,2.686,172.01000,0.02686
2021-10-26,173.36,185.00,172.50,177.84,177.84,2176749.0,2176749.0,4.48,2.584,178.44667,0.02584
2021-10-27,180.00,183.09,172.33,173.51,173.51,1106998.0,1106998.0,-6.49,-3.606,176.31000,-0.03606


In [25]:
## Make a function

In [51]:
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    return df

In [53]:
test = get_FMP_historical_data('TSLA')
test

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime
Date,,,,,,,,,,,
2020-01-02,424.500,430.70,421.710,430.26,86.052,9532100.0,9532100.0,5.760,1.357,427.55667,0.01357
2020-01-03,440.500,454.00,436.920,443.01,88.602,17778500.0,17778500.0,2.510,0.570,444.64333,0.00570
2020-01-06,440.470,451.56,440.000,451.54,90.308,10133000.0,10133000.0,11.070,2.513,447.70000,0.02513
2020-01-07,461.400,471.63,453.360,469.06,93.812,17882100.0,17882100.0,7.660,1.660,464.68333,0.01660
2020-01-08,473.700,498.49,468.230,492.14,98.428,31144300.0,31144300.0,18.440,3.893,486.28667,0.03893
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,950.530,1045.02,944.200,1024.86,1024.860,62852099.0,62852099.0,74.330,7.820,1004.69333,0.07820
2021-10-26,1024.690,1094.94,1001.440,1018.43,1018.430,62414968.0,62414968.0,-6.260,-0.611,1038.27000,-0.00611
2021-10-27,1039.660,1070.88,1030.780,1037.86,1037.860,38526459.0,38526459.0,-1.800,-0.173,1046.50667,-0.00173
